# Flow Run Management in Azure

**Requirements** - In order to benefit from this tutorial, you will need:
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed PromptFlow SDK


**Learning Objectives** - By the end of this tutorial, you should be able to:
- manage runs via run.yaml
- create run which references another runs inputs
- create run with connection override


**Motivations** - This guide will walk you through local run management abilities.

## 0. Install dependent packages

In [ ]:
%pip install -r ../../requirements.txt

## 1. Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

This notebook's will use connection `open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

In [ ]:
import json
from promptflow import PFClient
from promptflow.connections import AzureOpenAIConnection, OpenAIConnection

# client can help manage your runs and connections.
pf = PFClient()

In [ ]:
try:
    conn_name = "open_ai_connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<test_key>",
        api_base="<test_base>",
        api_type="azure",
        api_version="<test_version>",
    )

    # use this if you have an existing OpenAI account
    # connection = OpenAIConnection(
    #     name=conn_name,
    #     api_key="<user-input>",
    # )

    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

## 2. Create run with YAML file

You can save configurations for a run in a YAML file to save the effort to repeately provide them in SDK/CLI.
In this step, we will create a sample run with a YAML file. 

In [ ]:
from promptflow._sdk._load_functions import load_run

# load a run from YAML file
base_run = load_run(
    source="../../flows/standard/web-classification/run.yml",
    # override the default params in the YAML file
    params_override=[
        {"column_mapping": {"url": "${data.url}"}}
    ]
)

# create the run
base_run = pf.runs.create_or_update(run=base_run)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3 Create a flow run which uses an existing run's inputs data

When running a flow with an existing run, you can reference either it's inputs or outputs in column mapping.
The following code cell show how to reference a run's inputs in column mapping.

In [ ]:
from promptflow.entities import Run

# directly create the run object
run = Run(
    # local flow file
    flow="../../flows/standard/web-classification",
    # run name
    run=base_run,
    column_mapping={
        # reference another run's inputs
        "url": "${run.inputs.url}",
    },
)

base_run = pf.runs.create_or_update(
    run=run,
)

pf.runs.stream(base_run)

## 4. Create a flow run with connection override

Sometime you want to switch connection or deployment name inside a flow when submitting it.
Connection override provided an easy way to do it without changing original `flow.dag.yaml`.
In the following code cell, we will submit flow `web-classification` and override it's connection `open_ai_connection` to `azure_open_ai_connection`. 
Please make sure the connection `azure_open_ai_connection` exists in your local environment.

In [ ]:
run = Run(
    # local flow file
    flow="../../flows/standard/web-classification",
    data="../../flows/standard/web-classification/data.jsonl",
    # override connection for node classify_with_llm & summarize_text_content
    connections={
        "classify_with_llm": {"connection": "azure_open_ai_connection"},
        "summarize_text_content": {"connection": "azure_open_ai_connection"},
    },
)

base_run = pf.runs.create_or_update(
    run=run,
)

pf.runs.stream(base_run)